In [8]:
import os
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np

print(torch.__version__)
print(torchvision.__version__)
print(np.__version__)
print(sys.version)



2.5.1+cu124
0.20.1+cu124
2.1.3
3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]


In [ ]:
learning_rate = 1e-4
keep_prob_rate = 0.7  #
max_epoch = 3
BATCH_SIZE = 50


DOWNLOAD_MNIST = False
if not (os.path.exists("./data")) or not os.listdir("./data"):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True


train_data = torchvision.datasets.MNIST(
    root="./data",
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root="./data", train=False)
test_x = (
    Variable(torch.unsqueeze(test_data.test_data, dim=1), volatile=True).type(
        torch.FloatTensor
    )[:500]
    / 255.0
)
test_y = test_data.test_labels[:500].numpy()


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,  # input channel is 1
                out_channels=32,  # output channel is 32
                kernel_size=7,  # 7x7 conv kernel
                stride=1,  # stride=1 to move one pixel at a time
                padding=3,  # padding=(kernel_size-1)//2=3 to keep the same size
            ),  # output size is (batch_size, 32, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(2),  # maxpooling with 2x2 kernel
        )  # output size is (batch_size, 32, 14, 14)
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,  # input channel is 32
                out_channels=64,  # output channel is 64
                kernel_size=5,  # 5x5 conv kernel
                stride=1,  # stride=1 to move one pixel at a time
                padding=2,  # padding=(kernel_size-1)//2=2 to keep the same size
            ),  # output size is (batch_size, 64, 14, 14)
            nn.ReLU(), 
            nn.MaxPool2d(2),  # maxpooling with 2x2 kernel
        )   # output size is (batch_size, 64, 7, 7)
        self.out1 = nn.Linear(7 * 7 * 64, 1024, bias=True)

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024, 10, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #print(x.shape)
        x = x.view(
            x.size(0), -1
        )  # flatten the output of conv2 to (batch_size, 64 * 7 * 7)
        #print(x.shape)
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2, dim=1)  # 添加dim=1以确保正确计算softmax
        return output


def test(cnn):
    global prediction
    y_pre = cnn(test_x)
    _, pre_index = torch.max(y_pre, 1)
    pre_index = pre_index.view(-1)
    prediction = pre_index.data.numpy()
    correct = np.sum(prediction == test_y)
    return correct / 500.0


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x, y = Variable(x_), Variable(y_)
            output = cnn(x)  # forward propagation
            loss = loss_func(output, y)  # calculate loss
            optimizer.zero_grad()  # clear gradients for this training step
            loss.backward()  # backward propagation
            optimizer.step()  # update weights

            if step != 0 and step % 20 == 0:
                print(
                    "=" * 10,
                    step,
                    "=" * 5,
                    "=" * 5,
                    "test accuracy is ",
                    test(cnn),
                    "=" * 10,
                )


if __name__ == "__main__":
    cnn = CNN()
    train(cnn)
    

C:\Users\Dell\AppData\Local\Temp\ipykernel_26240\719360084.py:23: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  Variable(torch.unsqueeze(test_data.test_data, dim=1), volatile=True).type(


========== 20 ===== ===== test accuracy is  0.304 ==========
========== 40 ===== ===== test accuracy is  0.352 ==========
========== 60 ===== ===== test accuracy is  0.454 ==========
========== 80 ===== ===== test accuracy is  0.546 ==========
========== 100 ===== ===== test accuracy is  0.652 ==========
========== 120 ===== ===== test accuracy is  0.696 ==========
========== 140 ===== ===== test accuracy is  0.712 ==========
========== 160 ===== ===== test accuracy is  0.738 ==========
========== 180 ===== ===== test accuracy is  0.782 ==========
========== 200 ===== ===== test accuracy is  0.772 ==========
========== 220 ===== ===== test accuracy is  0.798 ==========
========== 240 ===== ===== test accuracy is  0.836 ==========
========== 260 ===== ===== test accuracy is  0.846 ==========
========== 280 ===== ===== test accuracy is  0.856 ==========
========== 300 ===== ===== test accuracy is  0.866 ==========
========== 320 ===== ===== test accuracy is  0.858 ==========
========== 3